In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import boto3

## 이전 작업에서 생성한 job_name을 `job_name` 변수에 입력하시기 바랍니다.

- `job_name`


In [ ]:
## From 1st notebook
job_name="YOUR_JOBNAME" 

## From 1st notebook
epochs = 10

## Log 확인을 위한 유틸 함수

### `job_name`을 이용하여 `logs` 스트림에서 해당 `job_name`의 마지막 이벤트 출력

1. `get_lsn` : Log 확인을 위한 lsn 값 반환 유틸 함수
2. `get_log` : Log 반환 유틸 함수
3. `parse_log` : Log 파서 유틸 함수

In [ ]:
## Log 확인을 위한 lsn 값 반환 유틸 함수
def get_lsn(job_name, lgn='/aws/sagemaker/TrainingJobs'):
    client = boto3.client('logs')
    response = client.describe_log_streams(
    logGroupName=lgn,
    orderBy='LastEventTime'
    )
    lsn = [ x['logStreamName'] for x in response['logStreams'] if x['logStreamName'].startswith(job_name)][0]
    return lsn 

## Log 반환 함수
def get_log(lsn, lgn='/aws/sagemaker/TrainingJobs'):
    client = boto3.client('logs')
    log=client.get_log_events(logGroupName=lgn, logStreamName=lsn)
    return log

## Log 파서
def parse_log(log):
    trn_accs=[]
    val_accs=[]
    for e in log['events']:
        msg=e['message']
        if 'Validation-accuracy' in msg:
            val = msg.split("=")
            val = val[1]
            val_accs.append(float(val))
        if 'Train-accuracy' in msg:
            trn = msg.split("=")
            trn = trn[1]
            trn_accs.append(float(trn))
    return trn_accs, val_accs

In [ ]:
lsn = get_lsn(job_name)
print(lsn)

In [ ]:
log = get_log(lsn)
trn_accs, val_accs = parse_log(log)

In [ ]:
print("Maximum validation accuracy: %f " % max(val_accs))
fig, ax = plt.subplots(figsize=(12,8))
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
trn_plot, = ax.plot(range(epochs), trn_accs, label="Training accuracy")
val_plot, = ax.plot(range(epochs), val_accs, label="Validation accuracy")
plt.legend(handles=[trn_plot,val_plot])
ax.yaxis.set_ticks(np.arange(0.4, 1.05, 0.05))
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%0.2f'))
plt.show()